In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 16

def fw_FDM(w, par):
    """
    Return the next time of the heat equation using forward difference method.
    
    Input:
        w: (1D array) Temperature distribution at time t.
        par: (dict) Parameter for the forward difference method.
    Output:
        w_new: (1D array) Temperature distribution at time t+1.
    """
    s = par['D']*par['k']/(par['h']*par['h'])

    w_new = np.zeros_like(w[1:-1])

    # Main line of the forward difference method
    w_new = w[1:-1] + s*(w[2:] - 2.*w[1:-1] + w[:-2])
    """
    This line is equivalent to the following for loop, but much faster.
    N = len(w)
    for i in range(1, N+1):
        w_new[i] = w[i] + s*(w[i-1] - 2*w[i] + w[i+1])
    """

    return w_new

In [2]:
def solve_heat_eq(ic, bc1, bc2, D, a, b, T, N, K, t0=0.):
    """
    Return numerical solution of Heat equation using forward difference method.
    
    Input:
        ic: (1D array) Initial temperature distribution.
        bc1: (float or 1D array) Boundary condition at x=a. Constant boundary condition is assumed.
        bc2: (float or 1D array) Boundary condition at x=b. Constant boundary condition is assumed.
        D: (float) Diffusion coefficient.
        a: (float) Left boundary.
        b: (float) Right boundary.
        T: (float) Final time.
        N: (int) Number of spatial grid points.
        K: (int) Number of time grid points.
        t0: (float) Initial time. (default=0.)
    Output:
        w: (2D array) Numerical solution.
    """
    w = np.zeros((N+2, K+1))
    h = (b-a)/(N+1)
    k = (T-t0)/K
    par = {'D': D, 'h': h, 'k': k}

    # Initial/boundary condition
    w[:, 0] = ic
    w[0, :] = bc1
    w[-1, :] = bc2

    # time marching
    for j in range(K):
        w[1:-1, j+1] = fw_FDM(w[:,j], par=par)

    return w

In [3]:
N = 30
K = 10
a = -1.
b = 1.
D = 1.

# Try:  T = 0.05 or bigger to see stability issue, 
#       T = 0.03 for stable solution
T = 0.05 
x = np.linspace(a, b, N+2)
t = np.linspace(0, T, K+1)

### Sanity Check 

Compute the numerical solution to (insert system here)

In [4]:
#%% Initial condition
# set initial condition
aa = 2.
CC = 2*np.pi/(b-a)
AA = aa*CC
f = lambda m, x: np.sin(AA*x) 
m = 0.

ic = f(m, x)
bc1 = f(m, a)
bc2 = f(m, b)

# solve heat equation
w = solve_heat_eq(ic, bc1, bc2, D, a, b, T, N, K)

# true solution
u_ = lambda x, t: np.exp(-AA*AA*t)*np.sin(AA*x)
u = u_(x.reshape(-1, 1), t)
"""The above line is equivalent to the following for loop, but much faster.
for j in range(K+1):
    u[:, j] = u_(x, t[j])
"""

'The above line is equivalent to the following for loop, but much faster.\nfor j in range(K+1):\n    u[:, j] = u_(x, t[j])\n'

In [5]:
#%% Initial condition
# set initial condition

f = lambda m, x: np.exp(-x*x/(2.*m))/np.sqrt(4.*np.pi*m)

m = 0.05
ic = f(m, x)
bc1 = f(m, a)
bc2 = f(m, b)

w = solve_heat_eq(ic, bc1, bc2, D, a, b, T, N, K)

In [ ]:
#%% plot
fig, ax = plt.subplots(1,1, figsize=(6.5, 6.5), subplot_kw={'projection':'3d'})

for j in range(K+1):
    ax.plot(x, t[j]*np.ones(N+2), w[:, j])

# Change the angle of projection

ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.view_init(elev=30, azim=45)

plt.show()